<a href="https://colab.research.google.com/github/hij826/PyTorchZeroToAll/blob/master/CifarToMnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.init

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' #cuda 사용할 때 쓰고 아니면 cpu 사용


In [0]:

trainset = dsets.MNIST(root='./data',
                       train=True,
                       download=True,
                       transform=transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=50,
                                          shuffle=True)

testset = dsets.MNIST(root='./data',
                      train=False,
                      download=True,
                      transform=transforms.ToTensor())
testloader = torch.utils.data.DataLoader(testset, batch_size=50,
                                         shuffle=False)


0it [00:00, ?it/s]

9920512it [00:01, 9304067.90it/s]                            


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 137811.46it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 2233214.34it/s]                            
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 52196.16it/s]            


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
training_epochs = 10

In [0]:
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding = 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding = 1)
        self.fc1 = nn.Linear(7*7*64, 120)
        self.fc2 = nn.Linear(120, 10)
        #self.fc3 = nn.Linear(84, 10)
        #torch.nn.init.xavier_uniform_(self.fc1.weight)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) #첫번째 layer통과
        x = self.pool(F.relu(self.conv2(x))) #두번째 layer통과
        x = x.view(-1, 7*7*64)
        x = self.fc1(x)
        #x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        #x = self.fc3(x)
        return x


net = Net().to(device)

In [0]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(net.parameters(), lr=0.002, momentum=0.9)

In [0]:
total_batch = len(trainloader)

for epoch in range(training_epochs):  # 데이터셋을 수차례 반복합니다.
    running_loss = 0
    
    for inputs,labels in trainloader:
        # 입력을 받은 후
        #inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # 변화도(Gradient) 매개변수를 0으로 만든 후
        optimizer.zero_grad()

        # 순전파 + 역전파 + 최적화
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 통계 출력
        running_loss += loss / total_batch
        #if i %  == 1999:    # print every 2000 mini-batches
    print('Epoch: {} loss: {}'. format(epoch + 1, running_loss))

print('Finished Training')

Epoch: 1 loss: 0.834699809551239
Epoch: 2 loss: 0.1675877422094345
Epoch: 3 loss: 0.10594495385885239
Epoch: 4 loss: 0.08267010003328323
Epoch: 5 loss: 0.0709725171327591
Epoch: 6 loss: 0.06293043494224548
Epoch: 7 loss: 0.056887686252593994
Epoch: 8 loss: 0.051379308104515076
Epoch: 9 loss: 0.04799375310540199
Epoch: 10 loss: 0.04512767866253853
Finished Training


In [0]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        labels = labels.to(device)
        outputs = net(images.to(device))
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 98 %


In [0]:
classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        labels = labels.to(device)
        outputs = net(images.to(device))
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of     0 : 98 %
Accuracy of     1 : 100 %
Accuracy of     2 : 100 %
Accuracy of     3 : 98 %
Accuracy of     4 : 97 %
Accuracy of     5 : 100 %
Accuracy of     6 : 100 %
Accuracy of     7 : 96 %
Accuracy of     8 : 100 %
Accuracy of     9 : 97 %
